In [1]:
import keras

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


In [31]:
model = keras.models.Sequential()
model.add(keras.layers.convolutional.RevCompConv1D(input_shape=(100,4), nb_filter=10, filter_length=11))
model.add(keras.layers.normalization.RevCompConv1DBatchNorm())
model.add(keras.layers.convolutional.RevCompConv1D(nb_filter=10, filter_length=11))
model.add(keras.layers.normalization.RevCompConv1DBatchNorm())
model.add(keras.layers.convolutional.RevCompConv1D(nb_filter=10, filter_length=11))
model.add(keras.layers.normalization.RevCompConv1DBatchNorm())
model.add(keras.layers.pooling.MaxPooling1D(pool_length=10))
model.add(keras.layers.convolutional.WeightedSum1D(symmetric=False, input_is_revcomp_conv=True, bias=False))
model.add(keras.layers.core.DenseAfterRevcompWeightedSum(output_dim=10))
model.add(keras.layers.core.Dense(output_dim=10))
model.compile(optimizer="sgd", loss="binary_crossentropy")

In [32]:
import numpy as np
rand_inp = np.random.random((10, 100, 4))

In [33]:
fwd_predict = model.predict(rand_inp)
rev_predict = model.predict(rand_inp[:, ::-1, ::-1])

In [35]:
np.max(np.abs(fwd_predict - rev_predict))

7.4505806e-08

In [36]:
#save and reload the model
json = model.to_json()
model.save_weights('model_weights.h5')
from keras.models import model_from_json
new_model = model_from_json(json)
new_model.load_weights('model_weights.h5')
fwd_predict_new = new_model.predict(rand_inp)
rev_predict_new = new_model.predict(rand_inp[:, ::-1, ::-1])
print(np.max(np.abs(fwd_predict - fwd_predict_new)))
print(np.max(np.abs(rev_predict - rev_predict_new)))

0.0
0.0
